# Это файл с решением задания №3

### Вопрос:
- По каждому предмету определи средний срок сдачи экзаменов
- (под сдачей понимаем последнее успешное прохождение экзамена студентом)

### Основная логика действий:

1. Создаем отдельный дата-сет для тех, кто сдавал экзамены (условие в задании: определи средний срок сдаачи экзаменов)
2. Удалим студента из предыдущего задания, кто не зарегался на экзамен
3. Мы не будем смотреть на пересдачников и тех, у кого оценка NA за экзамен, потому что в предыдущем задании их не было 
4. Поисследуем то, как связаны метрики date_submitted, date и module_presentation_length (все 3 обозначают дату и могут потенциально быть использованы для подсчета среднего срока сдачи экзаменов)
5. Определим средний срок сдачи экзаменов 

## Импорт либов 

In [3]:
import pandas as pd 

## Загрузка дата-сета

In [4]:
#Качаем дата-сеты
assesments = pd.read_csv('/home/jupyter-v-ankushev-18/Project_2_online/assessments.csv')
courses = pd.read_csv('/home/jupyter-v-ankushev-18/Project_2_online/courses.csv')
student_assesment = pd.read_csv('/home/jupyter-v-ankushev-18/Project_2_online/studentAssessment.csv')
student_registration = pd.read_csv('/home/jupyter-v-ankushev-18/Project_2_online/studentRegistration.csv')

#Связываем дата-сеты
courses_reg = pd.merge(courses,student_registration,on=['code_module', 'code_presentation'])
ocenki = pd.merge(courses_reg, assesments,on= ['code_module', 'code_presentation'])
full_ocenki = pd.merge(ocenki, student_assesment,on=['id_assessment', 'id_student'])

### Создаем дата-сет с теми, кто сдавал экзамен 

In [5]:
#Список критерий
examini = full_ocenki[full_ocenki['assessment_type'] == "Exam"].id_student.unique().tolist()

#Дата-сет с экзаменами
examin_data = full_ocenki[full_ocenki.id_student.isin(examini)]

### Удалим студента
кто не зарегистировался на экзамен, поскольку он не сдал экз –> он его дропнул

In [6]:
#Дропнутый студент
dropped_student = [425543]

#Удаление
examin_data_new = examin_data[~examin_data.id_student.isin(dropped_student)]

### Выбор метрик: 
Условие: сдача – это последнее успешное прохождение экзамена студентом 

- Это может быть date_submitted – дата сдачи теста студентом, измеряемая как количество дней с начала семестра.   
- Или может быть date – информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра.   Дата начала семестра имеет номер 0 (ноль)
- А может это быть module_presentation_length(продолжительность семестра в днях)

Давайте поисследуем, как связаны эти метрики (выявим их закономерности)

In [8]:
#1 - Сабсет студентов с успешно сданным экзаменом
#2 - группируем по предмету | семестру 
#3 - считаем value_counts для каждой из 3х дата-метрик: module_presentation_length, date, date_submitted

examin_data_new.query('score >= 40 & assessment_type == "Exam"') \
    [['code_module', 'code_presentation','id_student', 'module_presentation_length',
               'date', 'date_submitted']].groupby(['code_module', 'code_presentation']) \
                .agg({'module_presentation_length' : 'value_counts', 'date': 'value_counts','date_submitted': 'value_counts'})

module_presentation_length   date  date_submitted
CCC 2014B 230                         NaN    NaN           432.0
          234                         NaN    NaN           195.0
          236                         NaN    NaN            25.0
          237                         NaN    NaN             8.0
          238                         NaN    NaN             2.0
          241                       664.0    NaN             NaN
          242                         NaN    NaN             1.0
          266                         NaN    NaN             1.0
    2014J 243                         NaN    NaN           150.0
          244                         NaN    NaN           634.0
          245                         NaN    NaN           165.0
          248                         NaN    NaN            44.0
          249                         NaN    NaN             1.0
          250                         NaN    NaN            13.0
          251                         NaN    NaN             5.0
          252                         NaN    NaN             4.0
          259                         NaN    NaN             2.0
          269                      1019.0    NaN             NaN
          285                         NaN    NaN             1.0
DDD 2013B 229                         NaN    NaN           149.0
          230                         NaN    NaN           201.0
          231                         NaN    NaN           127.0
          233                         NaN    NaN            24.0
          240                       504.0  504.0             NaN
          241                         NaN    NaN             3.0
    2013J 237                         NaN    NaN           409.0
          241                         NaN    NaN           205.0
          242                         NaN    NaN           237.0
          243                         NaN    NaN            17.0
          244                         NaN    NaN             2.0
          245                         NaN    NaN             5.0
          248                         NaN    NaN             1.0
          250                         NaN    NaN             1.0
          255                         NaN    NaN             1.0
          261                       878.0  878.0             NaN
    2014B 234                         NaN    NaN           279.0
          236                         NaN    NaN           193.0
          237                         NaN    NaN             5.0
          238                         NaN    NaN             5.0
          241                       485.0  485.0             NaN
          242                         NaN    NaN             2.0
          251                         NaN    NaN             1.0
    2014J 242                         NaN    NaN           340.0
          243                         NaN    NaN           428.0
          244                         NaN    NaN            50.0
          245                         NaN    NaN            14.0
          248                         NaN    NaN             2.0
          249                         NaN    NaN             4.0
          250                         NaN    NaN             2.0
          256                         NaN    NaN             1.0
          262                       842.0    NaN             NaN
          279                         NaN    NaN             1.0

### Что видим:
1. Дата сдачи экзамена студентом (date_submitted) > выше, чем окончательная дата теста (date) и продолжительность семестра (module_presentation_length). Всего таких кейсов = 10    

2. Во многих семестрах мы не знаем дату окончательной сдачи (date). Но это неважно, потому что у нас есть данные с module_presentation_length. А module_presentation_length == date (т.е. если date == NA, то мы можем узнать ее значение из module_presentation_length).

То, что студенты сдавали экзамены до окончательной сдачи (date) и продолжительности семестра (module_presentation_length), это ок!

А вот обратная ситуация, странная. Давайте выделим этих студентов.

In [11]:
#Посмотрим на тех, у кого дата сдачи экзамена студентом (date_submitted) > чем module_presentation_length
(examin_data_new[examin_data_new['module_presentation_length'] < examin_data_new['date_submitted']]) \
    .query('assessment_type == "Exam" & score >= 40')

,code_module,code_presentation,module_presentation_length,id_student,date_registration,date_unregistration,id_assessment,assessment_type,date,weight,date_submitted,is_banked,score
49280,CCC,2014J,269,555498,-129.0,NaN,24299,Exam,NaN,100.0,285,0,58.0
58009,CCC,2014B,241,169380,-34.0,NaN,24290,Exam,NaN,100.0,242,0,78.0
60576,CCC,2014B,241,555008,-46.0,NaN,24290,Exam,NaN,100.0,266,0,56.0
80710,DDD,2014J,262,2341830,-151.0,NaN,25368,Exam,NaN,100.0,279,0,49.0
86405,DDD,2013B,240,532491,-120.0,NaN,25340,Exam,240.0,100.0,241,0,49.0
86845,DDD,2013B,240,538611,-82.0,NaN,25340,Exam,240.0,100.0,241,0,51.0
87853,DDD,2013B,240,548535,-24.0,NaN,25340,Exam,240.0,100.0,241,0,60.0
92030,DDD,2014B,241,391924,-93.0,NaN,25361,Exam,241.0,100.0,251,0,69.0
95003,DDD,2014B,241,631388,-15.0,NaN,25361,Exam,241.0,100.0,242,0,78.0
95302,DDD,2014B,241,634383,-22.0,NaN,25361,Exam,241.0,100.0,242,0,96.0


In [10]:
#Посмотрим на тех, у кого дата сдачи экзамена студентом (date_submitted) > чем date
(examin_data_new[examin_data_new['date'] < examin_data_new['date_submitted']])\
    .query('assessment_type == "Exam" & score >= 40')

,code_module,code_presentation,module_presentation_length,id_student,date_registration,date_unregistration,id_assessment,assessment_type,date,weight,date_submitted,is_banked,score
86405,DDD,2013B,240,532491,-120.0,NaN,25340,Exam,240.0,100.0,241,0,49.0
86845,DDD,2013B,240,538611,-82.0,NaN,25340,Exam,240.0,100.0,241,0,51.0
87853,DDD,2013B,240,548535,-24.0,NaN,25340,Exam,240.0,100.0,241,0,60.0
92030,DDD,2014B,241,391924,-93.0,NaN,25361,Exam,241.0,100.0,251,0,69.0
95003,DDD,2014B,241,631388,-15.0,NaN,25361,Exam,241.0,100.0,242,0,78.0
95302,DDD,2014B,241,634383,-22.0,NaN,25361,Exam,241.0,100.0,242,0,96.0


### Что видим?

1. Последние 6 студентов из первой таблицы – это те же самые люди, что во второй таблице
2. В условиях задания было сказано, что date имеет 0 в начале семестра. Нужно применить преобразование: добавить единичку!
+ Так как значения module_presentation_length те же, что и в date (если date != NA), то здесь мы тоже можем применить преобразование

In [13]:
#Посмотрим на тех, у кого дата сдачи экзамена студентом (date_submitted) > чем module_presentation_length
#С преобразованием!
(examin_data_new[examin_data_new['module_presentation_length'] + 1 < examin_data_new['date_submitted']]) \
    .query('assessment_type == "Exam" & score >= 40')

,code_module,code_presentation,module_presentation_length,id_student,date_registration,date_unregistration,id_assessment,assessment_type,date,weight,date_submitted,is_banked,score
49280,CCC,2014J,269,555498,-129.0,NaN,24299,Exam,NaN,100.0,285,0,58.0
60576,CCC,2014B,241,555008,-46.0,NaN,24290,Exam,NaN,100.0,266,0,56.0
80710,DDD,2014J,262,2341830,-151.0,NaN,25368,Exam,NaN,100.0,279,0,49.0
92030,DDD,2014B,241,391924,-93.0,NaN,25361,Exam,241.0,100.0,251,0,69.0


In [12]:
#Посмотрим на тех, у кого дата сдачи экзамена студентом (date_submitted) > чем date
#C преобразованием!
(examin_data_new[examin_data_new['date'] + 1 < examin_data_new['date_submitted']])\
    .query('assessment_type == "Exam" & score >= 40')

,code_module,code_presentation,module_presentation_length,id_student,date_registration,date_unregistration,id_assessment,assessment_type,date,weight,date_submitted,is_banked,score
92030,DDD,2014B,241,391924,-93.0,NaN,25361,Exam,241.0,100.0,251,0,69.0


### Что видим?
1. Студент из второй таблицы входит в первую. 
2. У 3х студентов date == NA 
3. У одного студента date == 241.0

Интерпретация второго пункта: 
- У студента была уважительная причина сдать экзамен позже, чем вся продолжительность семестра.   
Поэтому, учитель обозначил ему date = NA   
Например, студент заболел. Учитель назначил ему окончательную сдачу – NA. И он сдавал экзамен, когда выздоровеет

Интерпретация третьего пункта: 
Но у нас есть студент, кто имеет date - 241. Возможно, у него тоже была уважительная причина сдать экзамен позже!


Главный вопрос – включать этих студентов или нет?
Я считаю да, потому что они не выбросы, поскольку все-таки успешно сдали экзамен (не важно по каким причинам они опоздали по сдаче)!

Какую метрику выбираем?
Мы выбираем метрику date_submitted, поскольку она покрывает значения date и module_presentation_length   
(она самая информативная, широкая и имеет меньше всего NA)

### Определение среднего срока сдачи экзаменов по каждому предмету по метрике date_submitted

In [11]:
examin_data_new.query('score >= 40 & assessment_type == "Exam"') \
    .groupby('code_module', as_index = False).agg({'date_submitted' : 'mean'}).round(2)

,code_module,date_submitted
0,CCC,239.35
1,DDD,237.98


### Общий ответ на вопрос

- Для предмета CCC студенты в среднем сдавали экзамен через 239 дней с начала семестра 
- Для предмета DDD студенты в среднем сдавали экзамен через 238 дней с начала семестра 

Средний срок сдачи экзаменов для двух предметов практически не отличается!